# 주제분석

In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
review = pd.read_excel('review.xlsx',index_col=0)

In [4]:
review.head()

,review,label
0,맛있다! 편하다! 근성장!,1
1,임팩트 5kg 베타엑디스테론2통 LD한통 등등 구입했습니다~신타로 관리하다가...,1
2,가성비면에서는 말할것두 없구요 프로틴 세종류정도 먹어봤는데 이번 마이프로틴 스트...,1
3,가성비 때문에 먹기 시작했는데 입맛에 맞는 맛을 못찾아서 좀 방황하다가.. 저 ...,1
4,정말 말이 없는 최고 재품이네요~ 여러번 여러가지 맛을 먹어 봤지만 개인적인 취향...,1


In [5]:
import re
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

C:\Users\82107\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
token_re = re.compile(r'\b\w{2,}\b', re.UNICODE)
def tokenizer(text):
    text = text.lower()
    words = []
    for word in token_re.findall(text):
        if word not in ENGLISH_STOP_WORDS:
            words.append(word)
    return words

In [7]:
docs = []
for text in review.review:
    doc = tokenizer(text)
    docs.append(doc)

In [8]:
from gensim.corpora.dictionary import Dictionary

In [9]:
dic = Dictionary(docs)

In [10]:
dic.filter_extremes(no_below=10, no_above=0.9)

In [12]:
corpus = []
for doc in docs:
    bow = dic.doc2bow(doc)
    corpus.append(bow)

In [13]:
from gensim.models.ldamodel import LdaModel

In [14]:
from sklearn.model_selection import train_test_split
train_corpus, valid_corpus = train_test_split(corpus, test_size=0.1, random_state=0)

In [15]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [34]:
model = LdaModel(
    corpus=train_corpus,
    id2word=dic,
    num_topics=20,
    random_state=0)

In [35]:
loss = model.log_perplexity(valid_corpus)
loss

-11.792468426237793

In [36]:
import numpy
old_loss = -numpy.inf
while loss > old_loss + 0.1:
    model.update(train_corpus)
    old_loss = loss
    loss = model.log_perplexity(valid_corpus)
    print(loss)

-11.622559827543157
-11.50520830425349
-11.453595746907682


In [77]:
dic.token2id['보충제']

220

In [78]:
model.get_term_topics(220,0.01)

[(16, 0.02418731), (18, 0.013028249)]

In [79]:
model.show_topic(18)

[('정말', 0.040259123),
 ('최고입니다', 0.030865032),
 ('추천합니다', 0.028868206),
 ('추천', 0.027617455),
 ('초코브라우니', 0.024794847),
 ('맛을', 0.023458233),
 ('가성비', 0.020263383),
 ('풀리고', 0.018229151),
 ('먹으려고', 0.014818564),
 ('좋아하는', 0.014716374)]

In [40]:
import pyLDAvis.gensim

In [41]:
pyLDAvis.enable_notebook()

In [42]:
p = pyLDAvis.gensim.prepare(model, corpus, dic, sort_topics=False)

In [43]:
pyLDAvis.display(p)

In [27]:
from gensim.models import CoherenceModel

In [28]:
coh = CoherenceModel(model=model, corpus=corpus, texts=docs, dictionary=dic, coherence='c_v')

In [29]:
# 응집도
coh.get_coherence()

0.3746700156738946

In [30]:
# 다양도
# 상위 25개 
topn = 25
top_words = set()

for topic in range(model.num_topics):
    for word, prob in model.show_topic(topic, topn=topn):
        top_words.add(word)

In [31]:
len(top_words)

1261